## Part 1: Python for Data Processing

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Load the dataset
ts = pd.read_excel('traffic_stops (2).xlsx')

### Data Exploration and Cleaning

In [ ]:
print("Shape of dataset:", ts.shape)
print("\nColumns:", ts.columns.tolist())
ts.info()

# Drop columns that only contain missing values
ts.dropna(axis=1, how='all', inplace=True)

# Drop rows with any remaining missing values
ts.dropna(inplace=True)

# Verify that missing values are handled
print("\nMissing values after cleaning:")
print(ts.isnull().sum())

## Part 2: Database Design and Implementation

In [ ]:
# --- IMPORTANT: Replace with your database credentials ---
host = 'localhost'
port = '3306'
database = 'securecheck'
username = 'root'
password = 'your_password' # Replace with your password

connection_string = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

### Insert Cleaned Data into 'traffic_stops' Table

In [ ]:
# Using 'replace' will drop the table if it exists and create a new one.
# This is useful for rerunning the notebook without errors.
ts.to_sql(name='traffic_stops', con=engine, index=False, if_exists='replace')

print("✅ Data inserted successfully into 'traffic_stops' table!")

## Part 3: SQL Queries

#### 1. Gender distribution of drivers

In [ ]:
query1 = "SELECT driver_gender, COUNT(*) as count FROM traffic_stops GROUP BY driver_gender;"
gender_distribution = pd.read_sql(query1, engine)
display(gender_distribution)

#### 2. What is the most common violation for each driver race?

In [ ]:
query2 = """
WITH RankedViolations AS (
    SELECT
        driver_race,
        violation,
        COUNT(*) as violation_count,
        ROW_NUMBER() OVER(PARTITION BY driver_race ORDER BY COUNT(*) DESC) as rn
    FROM traffic_stops
    GROUP BY driver_race, violation
)
SELECT
    driver_race,
    violation,
    violation_count
FROM RankedViolations
WHERE rn = 1;
"""
common_violation_by_race = pd.read_sql(query2, engine)
display(common_violation_by_race)

#### 3. How does the arrest rate vary by time of day?

In [ ]:
query3 = """
SELECT
    CASE
        WHEN HOUR(stop_time) BETWEEN 6 AND 11 THEN 'Morning (6-12)'
        WHEN HOUR(stop_time) BETWEEN 12 AND 17 THEN 'Afternoon (12-18)'
        WHEN HOUR(stop_time) BETWEEN 18 AND 23 THEN 'Evening (18-24)'
        ELSE 'Night (0-6)'
    END as time_of_day,
    AVG(is_arrested) * 100 as arrest_rate_percent
FROM traffic_stops
GROUP BY time_of_day
ORDER BY arrest_rate_percent DESC;
"""
arrest_rate_by_time = pd.read_sql(query3, engine)
display(arrest_rate_by_time)